![EUROPE](http://nkpxv44v81c37wr753vdgve1.wpengine.netdna-cdn.com/wp-content/uploads/2015/10/Fotolia_44254321_Subscription_XXL-1000x250.jpg)

# WHERE TO RETIRE AT 50?

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

# Table of contents
1. [Research](#research)
2. [Data](#data)
    1. [Sub paragraph](#subparagraph1)
3. [Another paragraph](#paragraph2)


## Imports/Setup <a name="introduction"></a>

In [51]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

## RESEARCH <a name="research"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

## Data<a name="data"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

In [16]:
# Maths to convert longtitude and latitude to Mercator for displaying on map
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [17]:
# Testing Berlin Coordinates
merc('(52.52,13.40)')

(1491681.1766298658, 6894699.801282422)

## BOKEH EXAMPLE

In [14]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
# reset_output()
output_notebook()

Loading BokehJS ...

In [18]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON

#the data
xList=[1491681.1766298658]
yList=[6894699.801282422]
source=ColumnDataSource({'x':xList,'y':yList})

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
]

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-1000000, 3000000), y_range=(4000000, 9000000),
           x_axis_type="mercator", y_axis_type="mercator",width=900, height=600, tooltips=TOOLTIPS)
p.add_tile(CARTODBPOSITRON)

#plot a dot an a line
p.circle(source=source,x='x',y='y', size=10, color ='blue')#circle

show(p)